# Scrub Yelp dataset
Cleanup plan:

1. Remove records with no review text
2. Remove records with duplicated text
3. Fix referential integrity constraints

## Import dependencies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from time import time

## Read Yelp data

In [2]:
data_dir = '../../data/'
os.listdir(data_dir)

['yelp_public_business.csv',
 'corpus',
 'business.csv',
 'user.csv',
 'tip.csv',
 'review.csv',
 'yelp_public_yelp_user.csv',
 'checkin.csv']

In [3]:
t = time()
df_review = pd.read_csv(data_dir + 'review.csv')
elapsed = time() - t
print("----- DataFrame loaded"
      "\nin {0:.2f} seconds".format(elapsed) +
      "\nwith {0:,} rows\nand {1:,} columns"
      .format(df_review.shape[0], df_review.shape[1]) +
      "\n-- Column names:\n", df_review.columns)

----- DataFrame loaded
in 66.45 seconds
with 6,685,900 rows
and 9 columns
-- Column names:
 Index(['stars', 'review_id', 'user_id', 'funny', 'text', 'date', 'useful',
       'cool', 'business_id'],
      dtype='object')


In [4]:
t = time()
df_user = pd.read_csv(data_dir + 'yelp_public_yelp_user.csv')
elapsed = time() - t
print("----- DataFrame loaded"
      "\nin {0:.2f} seconds".format(elapsed) +
      "\nwith {0:,} rows\nand {1:,} columns"
      .format(df_user.shape[0], df_user.shape[1]) +
      "\n-- Column names:\n", df_user.columns)

/snap/jupyter/6/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


----- DataFrame loaded
in 22.96 seconds
with 1,577,383 rows
and 22 columns
-- Column names:
 Index(['fans', 'compliment_cute', 'useful', 'compliment_cool', 'yelping_since',
       'compliment_funny', 'review_count', 'average_stars', 'compliment_more',
       'funny', 'compliment_list', 'compliment_writer', 'friends',
       'compliment_hot', 'compliment_plain', 'name', 'elite',
       'compliment_photos', 'compliment_profile', 'compliment_note', 'user_id',
       'cool'],
      dtype='object')


In [5]:
t = time()
df_business = pd.read_csv(data_dir + 'yelp_public_business.csv')
elapsed = time() - t
print("----- DataFrame loaded"
      "\nin {0:.2f} seconds".format(elapsed) +
      "\nwith {0:,} rows\nand {1:,} columns"
      .format(df_business.shape[0], df_business.shape[1]) +
      "\n-- Column names:\n", df_business.columns)

/snap/jupyter/6/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


----- DataFrame loaded
in 2.86 seconds
with 192,506 rows
and 60 columns
-- Column names:
 Index(['attributes.Alcohol', 'hours', 'hours.Thursday', 'review_count',
       'stars', 'attributes.RestaurantsGoodForGroups', 'name',
       'attributes.BikeParking', 'hours.Tuesday', 'attributes.GoodForDancing',
       'attributes.Open24Hours', 'attributes.RestaurantsPriceRange2',
       'postal_code', 'attributes.Corkage', 'longitude',
       'attributes.AgesAllowed', 'hours.Friday', 'attributes.BYOBCorkage',
       'attributes.HappyHour', 'attributes.RestaurantsTableService', 'is_open',
       'hours.Sunday', 'attributes.WheelchairAccessible', 'attributes.WiFi',
       'attributes.AcceptsInsurance', 'state', 'attributes.CoatCheck',
       'attributes.BusinessAcceptsBitcoin', 'attributes.BestNights',
       'attributes.HairSpecializesIn', 'attributes.DietaryRestrictions',
       'attributes.OutdoorSeating', 'attributes.NoiseLevel',
       'attributes.RestaurantsAttire', 'attributes.GoodForMeal'

## Reviews with no text

In [6]:
df_review['text_len'] = df_review['text'].str.len()
mask1 = df_review['text_len'].isnull()
mask1.sum()

2

In [7]:
df_review[mask1]

,stars,review_id,user_id,funny,text,date,useful,cool,business_id,text_len
3427823,1.0,hpY467Acucmh3tCmEYGiDw,pUKAjAaSMkxiJ1WCIFSsBQ,2,NaN,2014-11-15 21:14:34,7,1,I7XWtKL1oHohZCtCCyQIog,NaN
6334066,3.0,iHdxK8lwkYn0rlNh-REuGQ,v3DIq18Z30KNwyUvBXtXEA,0,NaN,2014-12-26 08:32:06,0,0,oXoVJ0xKv82cBo9U6oEjlQ,NaN


In [8]:
df_review = df_review[~mask1]
print("Records with no review text were dropped. {0:,} records remain in the DataFrame."
      .format(len(df_review)))

Records with no review text were dropped. 6,685,898 records remain in the DataFrame.


## Duplicate check

In [9]:
mask1 = df_review.duplicated(subset=['text'], keep=False)
df_review.loc[mask1, 'text'].sort_values().head(10)

995610                                                     !
5053581                                                    !
4719326    !!!Times are changing and so is Pokedon!!!\n\n...
4701880    !!!Times are changing and so is Pokedon!!!\n\n...
1089545    "BBQ Brisket Hash" turned out to just be flavo...
1300803    "BBQ Brisket Hash" turned out to just be flavo...
2932293    "CARL STORGAARD"\n\nRemember this name as you ...
2963099    "CARL STORGAARD"\n\nRemember this name as you ...
2865313    "COOKING IS AN ART MASTERPIECES TAKE TIME" :)\...
2857689    "COOKING IS AN ART MASTERPIECES TAKE TIME" :)\...
Name: text, dtype: object

### Remove records with duplicate review text

In [10]:
mask1 = df_review.duplicated(subset=['text'])
mask1.sum()

17162

In [11]:
df_review = df_review[~mask1]
print("Records with duplicated text were dropped. {0:,} records remain in the DataFrame."
      .format(len(df_review)))

Records with duplicated text were dropped. 6,668,736 records remain in the DataFrame.


## Fix referential integrity constraints

### $\texttt{review}$ (FK: $\texttt{user_id}$) and $\texttt{user}$ (PK: $\texttt{user_id}$)

In [12]:
pk = 'user_id'
fk = 'user_id'
mask1 = df_review[fk].isin(df_user[pk])
len(df_review) - mask1.sum()

357285

In [13]:
t = time()
pk = 'user_id'
fk = 'user_id'
mask1 = df_review[fk].isin(df_user[pk])
old_len = len(df_review)
df_review = df_review[mask1]
elapsed = time() - t
print("Values of df2 not present in the Primary Key of df1 were dropped!"
      "\n{0:,} rows out of {1:,} were removed. {2:,} rows remaining."
      .format(old_len - len(df_review),
              old_len,
              len(df_review)),
      "\nTook {0:,.2f} seconds.".format(elapsed))

Values of df2 not present in the Primary Key of df1 were dropped!
357,285 rows out of 6,668,736 were removed. 6,311,451 rows remaining. 
Took 3.83 seconds.


### $\texttt{review}$ (FK: $\texttt{business_id}$) and $\texttt{business}$ (PK: $\texttt{business_id}$)

In [14]:
pk = 'business_id'
fk = 'business_id'
mask1 = df_review[fk].isin(df_business[pk])
len(df_review) - mask1.sum()

805

In [15]:
t = time()
pk = 'business_id'
fk = 'business_id'
mask1 = df_review[fk].isin(df_business[pk])
old_len = len(df_review)
df_review = df_review[mask1]
elapsed = time() - t
print("Values of df2 not present in the Primary Key of df1 were dropped!"
      "\n{0:,} rows out of {1:,} were removed. {2:,} rows remaining."
      .format(old_len - len(df_review),
              old_len,
              len(df_review)),
      "\nTook {0:,.2f} seconds.".format(elapsed))

Values of df2 not present in the Primary Key of df1 were dropped!
805 rows out of 6,311,451 were removed. 6,310,646 rows remaining. 
Took 2.34 seconds.


## Save results to a .csv file

In [17]:
save_path = data_dir + 'review_cleaned.csv'
t = time()
df_review.to_csv(save_path, index=False)
elapsed = time() - t
print("DataFrame saved to file:\n", save_path,
      "\ntook {0:.2f} seconds ({1:.2f} minutes)".format(elapsed, elapsed / 60))

DataFrame saved to file:
 ../../data/review_cleaned.csv 
took 160.20 seconds (2.67 minutes)
